In [8]:
#! pip install "snowflake-connector-python[pandas]"
# ! pip install scikit-learn

     |████████████████████████████████| 11.1MB 7.5MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 89.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/98/5d/5738903efe0ecb73e51eb44feafba32bdba2081263d40c5043568ff60faf/numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     |████████████████████████████████| 307kB 101.8MB/s eta 0:00:01


In [1]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session

/tmp/pip_packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
session = get_session()

#### Load the prepared data from snowflake

In [3]:
df = session.sql("SELECT * from {table}".format(table="predictive_maintenance_final"))

In [4]:
df.columns

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

Prepare dataset for training

In [5]:
df_final = df.to_pandas()

In [20]:
def trainer():
    #add all imports
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.metrics import recall_score, f1_score, roc_auc_score, confusion_matrix,classification_report
    from sklearn.linear_model import SGDClassifier
    from sklearn.pipeline import Pipeline
    
    df = session.sql("SELECT * from {table}".format(table="predictive_maintenance_final"))
    
    # prepare data for training
    X = df_final.drop(['FAILURE'], axis = 1)
    y = df_final['FAILURE']
    X_train, X_test, Y_train, Y_test = train_test_split(X,y,random_state = 0, test_size=0.25, shuffle = True)
    X_train.reset_index(inplace = True, drop = True)
    Y_train.reset_index(inplace = True, drop = True)

    X_test.reset_index(inplace = True, drop = True)
    Y_test.reset_index(inplace = True, drop = True)
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 0, test_size=0.25)
    
    pipeline = Pipeline([
        ('OHE', OneHotEncoder(handle_unknown = 'ignore',sparse=False)),
#         ('SCALER', StandardScaler()),
        ('SGD', SGDClassifier(random_state=0))
     ])
    
    pipeline.fit(x_train,y_train)
    y_pred = pipeline.predict(x_val) 
    cf_matrix = confusion_matrix(y_val, y_pred)
    cls_report = classification_report(y_val, y_pred, output_dict=True)
    print(cls_report)
    return pipeline, y_pred, cls_report

In [7]:
cls_report = trainer()

/tmp/pip_packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23323
           1       0.00      0.00      0.00        20

    accuracy                           1.00     23343
   macro avg       0.50      0.50      0.50     23343
weighted avg       1.00      1.00      1.00     23343



/tmp/pip_packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/pip_packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/pip_packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
type(cls_report)

str

In [25]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc

@sproc(name="predictive_maintenance", is_permanent=True, stage_location="@TRAINERS", replace=True, packages=["snowflake-snowpark-python","scikit-learn"])
def train_model(session: Session, features_table: str) -> dict:
    import os
    import time
    from joblib import dump
    start_time = time.time()
    df_final = session.sql("SELECT * from {table}".format(table="predictive_maintenance_final"))
    
    def trainer():
        #add all imports
        from sklearn.model_selection import train_test_split
        from sklearn.preprocessing import OneHotEncoder
        from sklearn.metrics import recall_score, f1_score, roc_auc_score, confusion_matrix,classification_report
        from sklearn.linear_model import SGDClassifier
        from sklearn.pipeline import Pipeline
    
    # prepare data for training
        X = df_final.drop(['FAILURE'], axis = 1)
        y = df_final['FAILURE']
        X_train, X_test, Y_train, Y_test = train_test_split(X,y,random_state = 0, test_size=0.25, shuffle = True)
        X_train.reset_index(inplace = True, drop = True)
        Y_train.reset_index(inplace = True, drop = True)

        X_test.reset_index(inplace = True, drop = True)
        Y_test.reset_index(inplace = True, drop = True)
    
        x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 0, test_size=0.25)
    
        pipeline = Pipeline([
            ('OHE', OneHotEncoder(handle_unknown = 'ignore',sparse=False)),
#         ('SCALER', StandardScaler()),
            ('SGD', SGDClassifier(random_state=0))
         ])
    
        pipeline.fit(x_train,y_train)
        y_pred = pipeline.predict(x_val) 
        cf_matrix = confusion_matrix(y_val, y_pred)
        cls_report = classification_report(y_val, y_pred, output_dict=True)
        print(cls_report)
        return pipeline, cls_report
    pipeline, cls_report = trainer()
    seconds = time.time() - start_time
    time_taken = str(time.strftime("%H:%M:%S",time.gmtime(seconds)))
    from datetime import datetime
    currentDateAndTime = datetime.now()
    model_filename = "Predictive_Maintenance_model_"+currentDateAndTime.strftime('%Y%m%d_%H%M%S') + '.joblib'
    
    model_file_path = os.path.join('/tmp',model_filename)
    dump(pipeline, model_file_path)
    
    session.file.put(model_file, "@MODELS" ,overwrite=True)
    
    return {"model_name":model_filename, "classification_report":cls_report}

In [26]:
test = train_model(session,"predictive_maintenance_final")

Failed to execute query [queryID: 01af1978-0503-caee-0072-f3030bf402d6] CALL predictive_maintenance('predictive_maintenance_final')
100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 7, in compute
  File "<ipython-input-25-31ef847b2466>", line 44, in train_model
  File "<ipython-input-25-31ef847b2466>", line 21, in trainer
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-packages/snowflake/snowpark/_internal/telemetry.py", line 184, in wrap
    r = func(*args, **kwargs)
TypeError: drop() got an unexpected keyword argument 'axis'
 in function PREDICTIVE_MAINTENANCE with handler compute


SnowparkSQLException: (1304): 01af1978-0503-caee-0072-f3030bf402d6: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 7, in compute
  File "<ipython-input-25-31ef847b2466>", line 44, in train_model
  File "<ipython-input-25-31ef847b2466>", line 21, in trainer
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-packages/snowflake/snowpark/_internal/telemetry.py", line 184, in wrap
    r = func(*args, **kwargs)
TypeError: drop() got an unexpected keyword argument 'axis'
 in function PREDICTIVE_MAINTENANCE with handler compute

In [18]:
session.call("predictive_maintenance",'predictive_maintenance_final')

Failed to execute query [queryID: 01af1973-0503-cad2-0072-f3030bf3e2b6] CALL predictive_maintenance('predictive_maintenance_final')
100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/32355341555881886/udf_py_394218412.zip/udf_py_394218412.py", line 6, in compute
    return func(session,arg1)
  File "<ipython-input-17-e84d19912585>", line 42, in train_model
  File "<ipython-input-17-e84d19912585>", line 36, in trainer
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-packages/sklearn/pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-package

SnowparkSQLException: (1304): 01af1973-0503-cad2-0072-f3030bf3e2b6: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/32355341555881886/udf_py_394218412.zip/udf_py_394218412.py", line 6, in compute
    return func(session,arg1)
  File "<ipython-input-17-e84d19912585>", line 42, in train_model
  File "<ipython-input-17-e84d19912585>", line 36, in trainer
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-packages/sklearn/pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/lib/python_udf/047598f675a8b237ddd7d3b635a81ce0d9bee24dad46b33e085ecf1fd433af7a/lib/python3.8/site-packages/sklearn/pipeline.py", line 370, in _fit
    X, fitted_transfor